# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [ ]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [ ]:
cities_df = pd.read_csv('../output_data/cities.csv')
cities_df.dropna(inplace = True) 
cities_df.head(20)

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [ ]:
gmaps.configure(api_key=g_key)

In [ ]:
# Store latitude and longitude in locations
locations = cities_df[["Lat", "Lng"]]

# Fill NaN values and convert to float
humidity = cities_df["Humidity"].astype(float)
maxi = humidity.max()

# Plot Heatmap
fig = gmaps.figure(center=(35.0, -30.0), zoom_level=2.4)

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=370,
                                 point_radius=5)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [ ]:
# Good conditions for golf
#
# A max temperature lower than 79 degrees but higher than 69.
# Wind speed less than 5 mph
# Humidity less than 40

cities_df.head(20)
hotel_df = cities_df[(cities_df['Max Temp'] < 89) & 
                        (cities_df['Max Temp'] > 69) & 
                        (cities_df['Wind Speed'] < 7) & 
                        (cities_df['Humidity'] < 40)
                       ]
hotel_df

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [ ]:
hotel_df['Hotel Name'] = ""
hotel_df

In [ ]:
# params dictionary to update each iteration
params = {
    "radius": 5000,
    "types": "hotels",
    "keyword": "hotel",
    "key": g_key
}

# Use the lat/lng we recovered to identify hotels
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # Use the search term: "hotel" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params)

    # convert to json
    name_address = name_address.json()
    # print(json.dumps(name_address, indent=4, sort_keys=True))

    # try-except to skip any that do not have a hotel nearby
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
hotel_df

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
hotel_layer = gmaps.marker_layer(locations, info_box_content=[f'{hotel}' for hotel in hotel_info])

fig = gmaps.figure(center=(30.0, -30.0), zoom_level=2)
fig.add_layer(heat_layer)
fig.add_layer(hotel_layer)

fig